In [ ]:
from numpy import genfromtxt
import matplotlib.pyplot as plt
#!pip install mplfinance
import mplfinance as mpf
from mplfinance.original_flavor import candlestick2_ochl, candlestick_ohlc
import numpy as np
import pandas as pd
import uuid
import time
import shutil
import os
import sys
from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras
from keras_OneCycle import OneCycle

#!pip install opencv-python
import cv2

%load_ext tensorboard

data = "data"
train = data + "/train/"
validation=  data + "/validation/"
buy = "buy/"
sell = "sell/"
hold = "hold/"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
### move x% of files to validation folder

val_pct = 0.2

for i in [buy, sell, hold]:
    files = os.listdir(train + i)
    
    # get index of only candles where signal occurs, a randome sample that is equal to number of buy/sell signals divided by 2 to keep a balance in trianing set:
    np.random.seed(42)
    files_to_move = np.random.choice(files, replace=False, size=round(len(files)*val_pct))

    #files_to_move = files[:round(len(files)*val_pct)]
    
    if not os.path.exists(validation+i):
        os.makedirs(validation+i)
    for f in files_to_move:
        shutil.move(train + i + f,validation + i) #change move to copy if you want to copy insted of moving 

In [ ]:
# If you like to speed up training process with GPU, first install PlaidML and then uncomment the following line.
# Otherwise it will fallback to tensorflow.

# os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

import tensorflow
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dropout, Flatten, Dense, Activation, BatchNormalization
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, Conv2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

DEV = True
argvs = sys.argv
argc = len(argvs)

if argc > 1 and (argvs[1] == "--development" or argvs[1] == "-d"):
  DEV = True

if DEV:
  epochs = 10
else:
  epochs = 1000

# BEFORE STARTING TRAINING YOU NEED TO MANUALLY TAKE 20 PERCENENT OF THE TRAINING DATA AND PUT IT INTO VALIDATION FOLDER
# I was too lazy to do it in the code.

train_data_dir = data + "/train/"
validation_data_dir = data + "/validation/"

# Input the size of your sample images
img_path = os.listdir(data + "/train/buy")[0]
im = cv2.imread(data + "/train/buy/" + img_path)

img_width = im.shape[0]
img_height = im.shape[1]


# Enter the number of samples, training + validation
nb_train_samples = len(os.listdir(train+buy)) + len(os.listdir(train+sell)) + len(os.listdir(train+"hold"))
nb_validation_samples = len(os.listdir(validation+buy)) + len(os.listdir(validation+sell)) + len(os.listdir(validation+"hold"))
nb_filters1 = 32
nb_filters2 = 32
nb_filters3 = 64
conv1_size = 3
conv2_size = 2
conv3_size = 2#5
pool_size = 2
# We have 3 classes, buy sell, and hold
classes_num = 3
batch_size = 32
lr = 0.001
chanDim =3



### custom model
model = Sequential()
model.add(Convolution2D(nb_filters1, conv1_size, conv1_size, padding ='same', input_shape=(img_height, img_width , 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

model.add(Convolution2D(nb_filters2, conv2_size, conv2_size, padding ="same"))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size), data_format='channels_last'))

model.add(Convolution2D(nb_filters3, conv3_size, conv3_size, padding ='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size), data_format='channels_last'))

model.add(Flatten())
model.add(Dense(1024))
model.add(Dense(1024))
model.add(Dense(1024))

model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(classes_num, activation='softmax'))

model.summary()
model.compile(loss='categorical_crossentropy',
                      optimizer=optimizers.RMSprop(),
                      metrics=['accuracy'])



train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    horizontal_flip=False)

test_datagen = ImageDataGenerator(
    rescale=1. / 255,
    horizontal_flip=False)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    #shuffle=True,
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    #shuffle=True,
    class_mode='categorical')

"""
Tensorboard log
"""
target_dir = "./models/weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
#if not os.path.exists(target_dir):
#  os.mkdir(target_dir)
model.save('./models/model.h5')
model.save_weights('./models/weights.h5')

logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
#callbacks_list = keras.callbacks.TensorBoard(log_dir=logdir)
callbacks_list = [
    OneCycle(min_lr=7e-6, max_lr=7e-2, min_mtm = 0.85, max_mtm = 0.95, annealing_stage=0.1, annealing_rate=0.01, training_iterations=np.ceil(((183*epochs)/(batch_size)))),
    keras.callbacks.ModelCheckpoint(
        "best_model.h5", save_best_only=True, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001
    ),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1),
]

#checkpoint = ModelCheckpoint(target_dir, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
#callbacks_list = [checkpoint]

training_history = model.fit(
    train_generator,
    steps_per_epoch=nb_train_samples//batch_size,
    epochs=epochs*10,
    shuffle=True,
    validation_data=validation_generator,
    callbacks=callbacks_list,
    validation_steps=nb_validation_samples//batch_size)

model.save("my_model")

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_30 (Conv2D)           (None, 120, 117, 32)      896       
_________________________________________________________________
activation_40 (Activation)   (None, 120, 117, 32)      0         
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 60, 58, 32)        0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 30, 29, 32)        4128      
_________________________________________________________________
activation_41 (Activation)   (None, 30, 29, 32)        0         
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 15, 14, 32)        0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 8, 7, 64)        

KeyboardInterrupt: 

In [ ]:
%tensorboard --logdir logs/scalars

In [ ]:
# Testing

test_data_dir = "data/test"

test_datagen = ImageDataGenerator(
    rescale=1. / 255,
    horizontal_flip=False)

test_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    #shuffle=True,
    class_mode='categorical')

Found 120 images belonging to 2 classes.


1e-07